AI Deep Learning – Simon Stijnen – May 2025

---

# Dinosaur Species Classification using Convolutional Neural Networks

This notebook implements a CNN model to classify dinosaur species using image data from Kaggle.

## Project Overview

In this project, we aim to build a deep learning model capable of distinguishing between 15 different dinosaur species using the [Dinosaur Image Dataset from Kaggle](https://www.kaggle.com/datasets/larserikrisholm/dinosaur-image-dataset-15-species).

The main objectives include:

1. Splitting the dataset into appropriate training, validation, and test sets
2. Selecting an appropriate CNN architecture
3. Tuning hyperparameters for optimal performance
4. Preventing overfitting with proper regularization techniques
5. Using Keras' Functional API to build the model
6. Evaluating the model with accuracy metrics and confusion matrices
7. Achieving an accuracy greater than 70%

In [ ]:
# Print python version
import sys
print("Python version:", sys.version)

## Dataset Preparation and Splitting

We'll use the `split-folders` library to properly split our dataset into training, validation, and test sets with a 70%-15%-15% ratio. This ensures we have proper separation for model evaluation and prevents data leakage.

In [ ]:
!pip install -q tensorflow scikit-learn matplotlib seaborn Pillow

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models, Input, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import os

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
!pip install -q kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("larserikrisholm/dinosaur-image-dataset-15-species")

print("Path to dataset files:", path)

In [ ]:
!pip install -q split-folders

import splitfolders

# Dataset path
input_folder = os.path.join(path, "dinosaur_dataset")  # Path to the dataset

# Create absolute path for output directory
output_split_dir = os.path.abspath(os.path.join("data", "dinosaur_dataset_split"))
os.makedirs(output_split_dir, exist_ok=True)

print("output_split_dir:", output_split_dir)

# Split dataset in train (70%), val (15%), test (15%)
splitfolders.ratio(
    input_folder,
    output=output_split_dir,
    seed=42,
    ratio=(0.7, 0.15, 0.15),
    group_prefix=None,
    move=False,
)

print("Dataset successfully split into train, validation, and test sets!")

## Data Configuration

Defining the path to the dataset and setting the image size and batch size for training, validation, and testing.

In [ ]:
# Dataset and model parameters
img_height, img_width = 296, 296
batch_size = 16

## Data Loading and Augmentation

In this section, we define the training, validation, and test datasets with appropriate data augmentation techniques for the training data.

In [ ]:
# Data preprocessing and augmentation for training data
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=applications.efficientnet.preprocess_input,
    brightness_range=[0.8, 1.2],  # Add brightness augmentation
    shear_range=0.1  # Add shear augmentation
)

# Only preprocessing for validation and test data (no augmentation)
val_test_datagen = ImageDataGenerator(
    preprocessing_function=applications.efficientnet.preprocess_input
)

# Load data from the split directories
train_data = train_datagen.flow_from_directory(
    f'{output_split_dir}/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_test_datagen.flow_from_directory(
    f'{output_split_dir}/val',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = val_test_datagen.flow_from_directory(
    f'{output_split_dir}/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,  # Don't shuffle for consistent evaluation
    class_mode='categorical'
)

print(f"Number of training samples: {train_data.samples}")
print(f"Number of validation samples: {val_data.samples}")
print(f"Number of test samples: {test_data.samples}")
print(f"Number of classes: {len(train_data.class_indices)}")

## Transfer Learning with EfficientNetB0

Instead of building a CNN from scratch, we'll use transfer learning with EfficientNetB0 as our base model. This model has been pre-trained on ImageNet and offers better accuracy with relatively low computational cost.

In [ ]:
# Create a base model from EfficientNetB0 - a more powerful CNN architecture with good efficiency
base_model = applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,  # Exclude the classification layer
    weights='imagenet'  # Use pre-trained weights from ImageNet
)

# Freeze the base model to prevent its weights from being updated during initial training
base_model.trainable = False

# Create our model by adding custom layers on top of the base model
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)  # Pass the inputs through the base model
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling reduces params & prevents overfitting

# Add more capacity to the top layers
x = layers.Dense(512, activation='relu')(x)  # Larger dense layer (512 instead of 256)
x = layers.BatchNormalization()(x)  # Add batch normalization for better training stability
x = layers.Dropout(0.4)(x)  # Slightly lower dropout rate 

# Add a second dense layer for more representation power
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(15, activation='softmax')(x)  # 15 classes output layer

model = models.Model(inputs, outputs)

# Compile the model with an appropriate learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # Start with a higher learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Add callbacks for better training
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'model/best_model_checkpoint.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# First phase: train just the top layers with the base model frozen
history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

## Fine-tuning the model

After initial training with the base model frozen, we can unfreeze some of the deeper layers of the base model and train them along with our custom top layers. This allows the model to fine-tune the pre-trained features to our specific dataset.

In [ ]:
import numpy as np
import tensorflow as tf

# EfficientNetB0 has more layers, so we'll gradually unfreeze
# First, unfreeze only the last 10% of the base model
total_layers = len(base_model.layers)
last_10_percent = int(total_layers * 0.1)

base_model.trainable = True

# Freeze all layers except the last 10%
for layer in base_model.layers[:-last_10_percent]:
    layer.trainable = False

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Use a lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print which layers are trainable
for layer in model.layers:
    print(f"{layer.name}: {layer.trainable}")

print(f"Total layers in base model: {total_layers}")
print(f"Unfreezing last {last_10_percent} layers for fine-tuning")

# Print counts of trainable and non-trainable parameters
trainable_count = np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights])

print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")
print(f"Total parameters: {trainable_count + non_trainable_count:,}")

# Display some trainable layers from the base model
print("\nLast 5 layers of base model and their trainable status:")
for layer in base_model.layers[-5:]:
    print(f"  {layer.name}: {layer.trainable}")

In [ ]:
# Calculate class weights to address class imbalance
from sklearn.utils.class_weight import compute_class_weight

# Get class distribution
class_counts = np.bincount(train_data.classes)

# Calculate class weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

# Convert to dictionary format required by Keras
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}
print("Class weights to handle imbalance:")
for class_name, class_idx in train_data.class_indices.items():
    print(f"  {class_name}: {class_weights[class_idx]:.2f}")

# Second phase: fine-tuning with unfrozen layers
fine_tune_history = model.fit(
    train_data,
    epochs=15,  # Increase epochs with early stopping
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weights  # Apply class weights
)

## Visualizing Training History

Let's plot the training and validation accuracy/loss to see how our model performed during training.

In [ ]:
# Combine histories from both training phases
acc = history.history['accuracy'] + fine_tune_history.history['accuracy']
val_acc = history.history['val_accuracy'] + fine_tune_history.history['val_accuracy']
loss = history.history['loss'] + fine_tune_history.history['loss']
val_loss = history.history['val_loss'] + fine_tune_history.history['val_loss']

plt.figure(figsize=(14, 6))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([9, 9], [0, 1], 'r--', label='Start Fine Tuning')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([9, 9], [0, 1], 'r--', label='Start Fine Tuning')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1.0])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

## Model Evaluation on Test Set

Now we'll evaluate our model on the completely separate test set to get a true measure of its performance.

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
# Make predictions on the test data
test_data.reset()  # Reset before predictions
y_pred_probs = model.predict(test_data)
y_pred = np.argmax(y_pred_probs, axis=1)

# Extract the true labels
# Since the test generator doesn't shuffle, labels align with class indices
y_true = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot raw counts confusion matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix (Counts)')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_labels, digits=3)
print("Classification Report:")
print(report)

# Calculate overall metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(f"Test Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision: {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall: {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1-Score: {f1_score(y_true, y_pred, average='macro'):.4f}")

## Sample Predictions

Let's visualize some sample predictions to see how our model performs on individual images.

In [ ]:
import random
from tensorflow.keras.preprocessing import image

def predict_and_display_image(img_path, model, class_labels):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    preprocessed_img = applications.efficientnet.preprocess_input(img_batch)
    
    # Make prediction
    predictions = model.predict(preprocessed_img)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Display image and prediction
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Predicted: {class_labels[predicted_class]}\nConfidence: {confidence:.2f}")
    plt.axis('off')
    plt.show()
    
    # Show top 3 predictions
    top_3_idx = np.argsort(predictions[0])[-3:][::-1]
    top_3_classes = [class_labels[i] for i in top_3_idx]
    top_3_confidences = [predictions[0][i] for i in top_3_idx]
    
    for cls, conf in zip(top_3_classes, top_3_confidences):
        print(f"{cls}: {conf:.4f}")

# Get a list of dinosaur classes
dino_classes = list(train_data.class_indices.keys())

# Define the test directory path
test_dir = os.path.join(output_split_dir, 'test')

# Sample a few images from different classes for prediction
for dino_class in random.sample(dino_classes, 3):
    class_dir = os.path.join(test_dir, dino_class)
    image_files = os.listdir(class_dir)
    sample_image = os.path.join(class_dir, random.choice(image_files))
    print(f"\nSample from class: {dino_class}")
    predict_and_display_image(sample_image, model, class_labels)

## Analyze Misclassifications

Let's examine some of the misclassified images to understand what might be confusing the model.

In [ ]:
# Get file paths and true labels from the test directory
test_image_paths = []
test_labels = []

for class_idx, class_name in enumerate(test_data.class_indices):
    class_dir = os.path.join(test_dir, class_name)
    for img_name in os.listdir(class_dir):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(class_dir, img_name)
            test_image_paths.append(img_path)
            test_labels.append(class_idx)

# Convert to numpy arrays
test_image_paths = np.array(test_image_paths)
test_labels = np.array(test_labels)

# Find misclassified indices
misclassified_indices = np.where(y_pred != y_true)[0]
print(f"Total misclassified images: {len(misclassified_indices)} out of {len(y_true)} ({len(misclassified_indices)/len(y_true)*100:.2f}%)")

# Display random misclassified images
n_display = min(6, len(misclassified_indices))
sample_indices = np.random.choice(misclassified_indices, n_display, replace=False)

plt.figure(figsize=(15, 10))
for i, idx in enumerate(sample_indices):
    # Get the file path for this test image
    img_idx = idx % len(test_image_paths)  # Handle case where idx is out of range
    img_path = test_image_paths[img_idx]
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    preprocessed_img = applications.efficientnet.preprocess_input(img_batch)
    
    # Get predictions
    predictions = model.predict(preprocessed_img)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Get true class
    true_class = y_true[idx]
    
    # Get top 3 predictions for this image
    top_3_idx = np.argsort(predictions[0])[-3:][::-1]
    top_3_classes = [class_labels[i] for i in top_3_idx]
    top_3_confidences = [predictions[0][i] for i in top_3_idx]
    top_3_text = '\n'.join([f"{cls}: {conf:.2f}" for cls, conf in zip(top_3_classes, top_3_confidences)])
    
    # Plot
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(f"True: {class_labels[true_class]}\nPred: {class_labels[predicted_class]}\nConf: {confidence:.2f}")
    plt.xlabel(top_3_text)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Analyze which classes are most frequently confused
confusion_pairs = [(class_labels[y_true[i]], class_labels[y_pred[i]]) for i in misclassified_indices]
pairs_count = {}
for true_class, pred_class in confusion_pairs:
    if (true_class, pred_class) in pairs_count:
        pairs_count[(true_class, pred_class)] += 1
    else:
        pairs_count[(true_class, pred_class)] = 1

# Get the top 5 most confused pairs
top_confused_pairs = sorted(pairs_count.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop confused class pairs (true → predicted):")
for (true_class, pred_class), count in top_confused_pairs:
    print(f"  {true_class} → {pred_class}: {count} times")

In [ ]:
# Save the model architecture and weights
model.save('model/dinosaur_classifier_transfer_learning.keras')

# Save the class labels mapping
import json
with open('model/dinosaur_class_mapping.json', 'w') as f:
    json.dump(test_data.class_indices, f)

# Save test performance metrics
test_metrics = {
    'accuracy': float(accuracy_score(y_true, y_pred)),
    'precision': float(precision_score(y_true, y_pred, average='macro')),
    'recall': float(recall_score(y_true, y_pred, average='macro')),
    'f1_score': float(f1_score(y_true, y_pred, average='macro')),
    'classes': test_data.class_indices
}

with open('model/dinosaur_model_performance.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

print("Model, class mapping, and performance metrics saved successfully!")